In [1]:
"""
File : data_clean_DAL.ipynb
Created by : Amen Ouannes
Brief : preparing the data from DAL sensor to be used to the training

"""

'\nFile : data_clean_DAL.ipynb\nCreated by : Amen Ouannes\nBrief : preparing the data from DAL sensor to be used to the training\n\n'

In [3]:
#import libraries
import data_O as tool
import os as os
import pandas as pd
import pickle as pk
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from dateutil import tz
print(os.getcwd())

/data/amen/amen_env/data_processing


In [4]:
#extract data from DAL sensors
data = pd.read_csv("/data/amen/amen_env/data_processing/data/visits_point.csv", sep =';')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24124 entries, 0 to 24123
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   calfNumber         24124 non-null  int64  
 1   feedingDay         24124 non-null  int64  
 2   feeder             24124 non-null  int64  
 3   feederLong         24124 non-null  object 
 4   initial            24124 non-null  object 
 5   group              24124 non-null  object 
 6   station            15127 non-null  float64
 7   Station            24124 non-null  int64  
 8   date               24124 non-null  object 
 9   Activity           24124 non-null  int64  
 10  ActivityAfter      24124 non-null  int64  
 11  ActivityDrinking   24124 non-null  int64  
 12  Additive_1         24124 non-null  int64  
 13  Additive_2         24124 non-null  int64  
 14  Consumption        24124 non-null  int64  
 15  correctionDays     24124 non-null  int64  
 16  Duration           241

In [13]:
#trunc data for only the calves whom will be used for the study
data = data[data['calfNumber'].isin([8854, 8851, 7480, 8855, 8846])]

In [14]:
#reduce useless columns
data.drop(['Activity', 'ActivityAfter', 'ActivityDrinking','initial',
          'Additive_1', 'Additive_2', 'Electrolyte', 'date', 'FITActiveTomorrow',
          'ScaleValid', 'ScaleValidFlag', 'Weight', 'group','feeder', 'feederLong',
           'feedingDay', 'correctionDays',
           'station', 'Station', '_id', 'calf'], axis = 1, inplace = True)
print(data.shape)

(2770, 6)


In [5]:
#Dropping all the passing-by without consumption data, could get rid of the type column
data = data[data['Type'] =='wEnt']
data.drop(['Type'], axis = 1, inplace = True)
print(data)

       calfNumber  Consumption  Duration  Entitlement            localDate
1            8846          550       491         2450  2024-04-07T19:55:29
55           8846          550       423         3000  2024-04-07T18:45:41
66           7480         3000       674         3000  2024-04-07T18:32:27
119          8855         2000       662         3000  2024-04-07T16:32:11
163          8855            0       374         3000  2024-04-07T15:37:54
...           ...          ...       ...          ...                  ...
23417        8855         1665       561         1665  2024-05-21T08:04:50
23613        8855          840       140          840  2024-05-20T20:27:20
23757        8855         1680       416         1680  2024-05-20T15:13:45
23930        8855         1680       208         1680  2024-05-20T07:24:35
24108        8855         1200       206         1200  2024-05-19T20:22:32

[1419 rows x 5 columns]


In [15]:
#converting the localDate to timestamp for display purposes
print(type(data.iloc[1]['localDate']))
data['localDate'] = pd.to_datetime(data['localDate'])
print(type(data.iloc[1]['localDate']))

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [16]:
#let's take the data only between 4 and 10th May
dt1 = "2024-05-04T00:00:00"
dt2 = "2024-05-11T00:00:00"
data.sort_values(by='localDate', inplace=True)
# Filter the DataFrame to include rows between dt1 and dt2
data = data[(data['localDate'] >= dt1) & (data['localDate'] < dt2)]
print(data)

       calfNumber  Consumption  Duration  Entitlement           localDate  \
15239        8855            0         0            0 2024-05-04 00:24:30   
15233        8855            0         0            0 2024-05-04 00:42:09   
15226        8854            0         0            0 2024-05-04 01:10:37   
15225        8854            0         0            0 2024-05-04 01:21:52   
15224        8855            0         0            0 2024-05-04 02:27:51   
...           ...          ...       ...          ...                 ...   
17717        8846            0         0            0 2024-05-10 21:30:56   
17712        7480            0         0            0 2024-05-10 21:40:21   
17705        8851            0         0            0 2024-05-10 22:10:00   
17704        8855            0         0            0 2024-05-10 22:27:37   
17703        8846            0         0            0 2024-05-10 23:16:27   

        Type  
15239  woEnt  
15233  woEnt  
15226  woEnt  
15225  woEnt  


In [8]:
#Eliminate stops without drinking:
data = data[data['Consumption'] != 0]

In [17]:
#Calculate the full duration of drinking
Data_span = data['Duration'].sum()
Data_mean = data['Duration'].mean()
print(f"{Data_span//(5)} total of sequences, {Data_mean//5} drinking sequences per visit")

38650 total of sequences, 63.0 drinking sequences per visit


In [18]:
#Create a new dataframe that contains the true values of drinking for the training
DAL = pd.DataFrame(columns = ['time','calfNumber', 'state'])
for j in range(len(data)):
    duration = (data.iloc[j]['Duration'])
    time = data.iloc[j]['localDate']
    calf = data.iloc[j]['calfNumber']
    #looping for the duration of one drink event with 2 seconds intervals
    for seconds in range(0, duration, 1):  
        new_row = pd.DataFrame([
            {'time': time + timedelta(seconds=seconds),
             'calfNumber' : calf, 'state': 1}
        ])
        DAL = pd.concat([DAL, new_row], ignore_index=True)

        
DAL.shape
print(DAL)

/tmp/ipykernel_959191/2252559877.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  DAL = pd.concat([DAL, new_row], ignore_index=True)


                      time calfNumber state
0      2024-05-04 04:13:15       8851     1
1      2024-05-04 04:13:16       8851     1
2      2024-05-04 04:13:17       8851     1
3      2024-05-04 04:13:18       8851     1
4      2024-05-04 04:13:19       8851     1
...                    ...        ...   ...
193248 2024-05-10 21:03:34       8855     1
193249 2024-05-10 21:03:35       8855     1
193250 2024-05-10 21:03:36       8855     1
193251 2024-05-10 21:03:37       8855     1
193252 2024-05-10 21:03:38       8855     1

[193253 rows x 3 columns]


In [19]:
tool.download(DAL, "DAL.pkl")